<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/pyenv-virtualenv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.22.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2024.9.0 requires numpy>=1.24, but you have numpy 1.23.5 which i

In [1]:
!pip install insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 27.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 99.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055393 sha256=8617f6d455f109fb9006081c7bad885510121c0be200d53295244f68a85174dc
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface


In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00


In [9]:
import cv2
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis

In [10]:
class INSIGHTFACE(object):
    def __init__(self, ctx_id=0, det_size_x=640, det_size_y=640):
        self.app = FaceAnalysis(root='./.insightface')
        self.app.prepare(ctx_id=ctx_id, det_size=(det_size_x, det_size_y))
        self.faces = []

    def get_InsightFace(self, img):
        self.faces = self.app.get(np.asarray(img))
        #for face in faces:
        #    kps   = face.kps.astype(np.int)
        #    print(kps)
        rimg  = self.app.draw_on(img, self.faces)
        #cv2.imwrite("./output.jpg", rimg)
        return img, rimg, self.faces

    def draw_mesen(self, img):
        KPS = []
        for face in self.faces:
            kps = face.kps.astype(np.int)
            KPS.append(kps)
        EYE = _EYE = [[]] * 2
        out_img = img
        for _eye in KPS:
            EYE[0] = _eye[0]
            EYE[1] = _eye[1]
            lineSize  = abs(np.linalg.norm(np.array(EYE[0])-np.array(EYE[1])))
            lineWidth = int(lineSize / 4)
            if lineWidth <= 10:
                lineWidth = 10
            lineAngle = 1
            if EYE[1][0]-EYE[0][0] != 0:
                lineAngle   = (EYE[1][1]-EYE[0][1])/(EYE[1][0]-EYE[0][0])
            _EYE[0][0] = EYE[0][0] - (lineSize / 3)
            _EYE[0][1] = lineAngle * (_EYE[0][0] - EYE[0][0]) + EYE[0][1]
            _EYE[1][0] = EYE[1][0] + (lineSize / 3)
            _EYE[1][1] = lineAngle * (_EYE[1][0] - EYE[0][0]) + EYE[0][1]
            out_img = cv2.line(out_img,
                              (_EYE[0][0], _EYE[0][1]),
                              (_EYE[1][0], _EYE[1][1]),
                              (0,0,0),
                              thickness=lineWidth)
        return out_img

In [11]:
IF = INSIGHTFACE()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [12]:
input_path = './zoom_480p.mov'
output_path = './zoom_480p_mask.mp4'
cap = cv2.VideoCapture(input_path)

In [13]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = float(cap.get(cv2.CAP_PROP_FPS))
frameNum = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [14]:
codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_path, codec, fps, (width, height), True)

In [16]:
bar = tqdm(total=frameNum, dynamic_ncols=True)
ret, img = cap.read()
print(ret)
while ret :
    input_img, draw_img, faces = IF.get_InsightFace(img)
    if len(faces):
        out_img = IF.draw_mesen(input_img)
        #input_img, draw_img, faces = IF.get_InsightFace(input_img)  #なぜか、2回処理すると取りこぼしがなくなる。
        #out_img = IF.draw_mesen(out_img)
    ret, img = cap.read()
    bar.update(1)
writer.release()


  1%|          | 210/38882 [00:51<2:37:58,  4.08it/s]


True


<ipython-input-10-361056217de6>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kps = face.kps.astype(np.int)

  0%|          | 27/38882 [00:06<2:58:51,  3.62it/s]

KeyboardInterrupt: 

In [13]:
! ls -alh

total 633M
drwxr-xr-x 1 root root 4.0K Oct 18 08:06 .
drwxr-xr-x 1 root root 4.0K Oct 18 07:54 ..
drwxr-xr-x 4 root root 4.0K Oct 14 13:22 .config
drwxr-xr-x 2 root root 4.0K Oct 18 08:01 .ipynb_checkpoints
drwxr-xr-x 1 root root 4.0K Oct 14 13:23 sample_data
-rw-r--r-- 1 root root 244M Oct 18 08:07 zoom_480p_mask.mp4
-rw-r--r-- 1 root root 390M Oct 18 08:05 zoom_480p.mov


In [14]:
!cp ./zoom_480p_mask.mp4 ./drive/MyDrive